In [1]:
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# from utils_subdivision.gen_distribution_single_plots import analyze_phases
# from utils_subdivision.gen_distribution_subplot import analyze_single_type    # plot_combined_results
# from utils_subdivision.gen_distribution_merged_plot import plot_merged
from utils_dot_plot.drum_single import analyze_phases
# from utils_dot_plot.drum_merged import plot_merged_per_mode

from utils_subdivision.gen_distribution_subplot import analyze_single_type
from utils_dot_plot.kinematic_dot_plot import *
from utils_dot_plot.drum_merged import *


base_output_dir = "output_dot_plots"

# Merged Drum Dot Plot Per Piece

In [ ]:
# # # Main execution code --------------------------------------------------------------------
# m_idx = 2
# mode = ["group", "individual", "audience"]
# dance_mode = mode[m_idx]

# # Dictionary to store drum_phases_kde for each piece
# piece_drum_phases_kde = {}

# with open('data/selected_piece_list.pkl', 'rb') as f:
#     piece_list = pickle.load(f)

# # Collect data for all pieces
# for file_name in piece_list:
#     print(file_name)
    
#     # Determine piece type
#     PIECE_TYPES = ["Suku", "Maraka", "Manjanin", "Wasulunka"]
#     piece_type = next((p for p in PIECE_TYPES if p in file_name), None)
#     if piece_type is None:
#         print(f"Warning: Unknown piece type for {file_name}")
#         continue
    
#     # Load data paths
#     cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
#     onsets_csv_path = f"data/drum_onsets/{file_name}.csv"
#     dmode_path = f"data/dance_modes_ts/{file_name}_{dance_mode}.pkl"
    
#     if os.path.exists(dmode_path):
#         with open(dmode_path, "rb") as f:
#             dance_mode_time_segments = pickle.load(f)
#     else:
#         continue
    
#     # Get drum phases and KDE data
#     _, _, drum_phases_kde = plot_merged_stacked(
#         file_name=file_name,
#         dance_mode=dance_mode,
#         cycles_csv_path=cycles_csv_path,
#         onsets_csv_path=onsets_csv_path,
#         dance_mode_time_segments=dance_mode_time_segments,
#         figsize=(10, 3),
#         dpi=200,
#         use_window=True
#     )
    
#     # Store the data
#     if piece_type not in piece_drum_phases_kde:
#         piece_drum_phases_kde[piece_type] = []
#     piece_drum_phases_kde[piece_type].append(drum_phases_kde)

# save_path = os.path.join(base_output_dir, f"piece_drum_phases_kde_{dance_mode}.pkl")
# with open(save_path, 'wb') as f:
#     pickle.dump(piece_drum_phases_kde, f)
# print(f"\nSaved piece KDE data to: {save_path}")

In [9]:
def plot_combined_merged_stacked(piece_type, dance_mode, drum_phases_kde_all, figsize=(10, 3), dpi=200):
    """Create a single plot showing combined merged analysis for all pieces of a type"""
    
    # Set up single merged axes
    fig, ax = plt.subplots(figsize=figsize, dpi=dpi)
    onset_types = ['Dun', 'J1', 'J2']
    colors = ['#1f77b4', '#2ca02c', '#d62728']  # Blue, Green, Red
    
    # vertical stacking ranges for each type
    vertical_ranges = {
        'Dun': (1, 6),    # y from 1–6
        'J1':  (8, 13),   # y from 8–13
        'J2':  (15, 20),  # y from 15–20
    }

    combined_h = None
    combined_xx = None

    # Combine data for each onset type
    for onset_type, color in zip(onset_types, colors):
        # Combine phases and y_scaled from all pieces
        all_phases = []
        all_y_scaled = []
        segment_kde_h = None
        segment_kde_xx = None

        # Loop through all pieces' data
        for piece_data in drum_phases_kde_all:
            if onset_type in piece_data:
                # Combine phases and y_scaled
                all_phases.extend(piece_data[onset_type]["phases"])
                all_y_scaled.extend(piece_data[onset_type]["y_scaled"])
                
                # Accumulate KDE
                if segment_kde_h is None:
                    segment_kde_h = piece_data[onset_type]["kde_h"].copy()
                    segment_kde_xx = piece_data[onset_type]["kde_xx"].copy()
                else:
                    segment_kde_h += piece_data[onset_type]["kde_h"]

        if not all_phases:  # Skip if no data found
            continue

        # Convert lists to numpy arrays
        phases = np.array(all_phases)
        y_scaled = np.array(all_y_scaled)
            
        # Plot scatter with single color for each instrument
        ax.scatter(phases * 400,
                   y_scaled,
                   s=5, alpha=0.6,
                   color=color,
                   label=onset_type)

        # accumulate KDE
        if combined_h is None:
            combined_h = segment_kde_h.copy()
            combined_xx = segment_kde_xx.copy()
        else:
            combined_h += segment_kde_h

    # Draw combined KDE at bottom (-5 to 0)
    if combined_h is not None:
        # normalize to 0–1
        combined_h = combined_h / np.max(combined_h)
        kde_scaled = -5 + (5 * combined_h)
        
        ax.fill_between(combined_xx * 400,
                        -5, kde_scaled,
                        alpha=0.3, color='purple',
                        label='Combined KDE')

    # Add subdivision lines
    for subdiv in range(1, 13):  # 12 subdivisions
        color = get_subdiv_color(subdiv)
        x_pos = ((subdiv-1) * 400) / 12
        
        if subdiv in [1, 4, 7, 10]:
            ax.vlines(x_pos, -5.5, 20.5, color=color, linestyle='-', linewidth=1.5, alpha=0.7)
        else:
            ax.vlines(x_pos, -5.5, 20.5, color=color, linestyle='--', linewidth=1, alpha=0.3)

    # Styling & axes
    xtick = [0, 100, 200, 300, 400]
    xtick_labels = [1, 2, 3, 4, 5]
    
    ax.set_xlim(0, 400)
    ax.set_xticks(xtick)
    ax.set_xticklabels(xtick_labels)
    ax.set_xlabel('Beat span')
    
    ax.set_ylim(-5.5, 20.5)
    ax.set_yticks([3, 10, 17])
    ax.set_yticklabels(['Dun', 'J1', 'J2'])
    
    ax.set_ylabel('Instrument')
    ax.grid(True, alpha=0.3)
    ax.set_xlim(-33, 400)

    # Title & legend
    title = f'Piece Type: {piece_type} | Dance Mode: {dance_mode}'
    title += f' | Combined from {len(drum_phases_kde_all)} pieces'
    ax.set_title(title, pad=10)
    
    # Add legend
    ax.legend(loc='upper left', framealpha=0.4, fontsize=6)

    return fig, ax

In [10]:

m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]
save_path = os.path.join(base_output_dir, f"piece_drum_phases_kde_{dance_mode}.pkl")


with open(save_path, 'rb') as f:        
    piece_drum_phases_kde = pickle.load(f)      # This was saved in the previous cell
    
    
PIECE_TYPES = ["Suku", "Maraka", "Manjanin", "Wasulunka"]
# Create combined plots for each piece type
for piece_type in PIECE_TYPES:
    if piece_type in piece_drum_phases_kde:
        fig, ax = plot_combined_merged_stacked(
            piece_type=piece_type,
            dance_mode=dance_mode,
            drum_phases_kde_all=piece_drum_phases_kde[piece_type]
        )
        
        
        # plt.show()
        # Save the figure
        save_dir = os.path.join(base_output_dir, "drum_kde_by_piece", dance_mode)
        os.makedirs(save_dir, exist_ok=True)
        
        save_path = os.path.join(save_dir, f"{piece_type}_{dance_mode}_combined.png")
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()

# Generate Dance dot plot per piece

In [ ]:
# # Main execution code --------------------------------------------------------------------
# m_idx = 0
# mode = ["group", "individual", "audience"]
# dance_mode = mode[m_idx]

# # Dictionary to store dance_phases_kde for each piece type
# piece_dance_phases_kde = {}

# with open('data/selected_piece_list.pkl', 'rb') as f:
#     piece_list = pickle.load(f)

# # Collect data for all pieces
# for file_name in piece_list:
#     print(file_name)
    
#     # Determine piece type
#     piece_type = next((p for p in PIECE_TYPES if p in file_name), None)
#     if piece_type is None:
#         print(f"Warning: Unknown piece type for {file_name}")
#         continue
    
#     # Load data paths
#     cycles_csv_path = f"data/virtual_cycles/{file_name}_C.csv"
#     left_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_left_foot_onsets.csv"
#     right_onset_path = f"data/logs_v4_0.007_foot_jun3/{file_name}_T/onset_info/{file_name}_T_right_foot_onsets.csv"
#     dmode_path = f"data/dance_modes_ts/{file_name}_{dance_mode}.pkl"
    
#     left_onsets = pd.read_csv(left_onset_path)["time_sec"].values
#     right_onsets = pd.read_csv(right_onset_path)["time_sec"].values
    
#     if os.path.exists(dmode_path):
#         with open(dmode_path, "rb") as f:
#             dance_mode_time_segments = pickle.load(f)
#     else:
#         continue
    
#     # Get foot phases and KDE data
#     fig, ax, dance_phases_kde = plot_foot_onsets_stacked(
#         file_name=file_name,
#         dance_mode=dance_mode,
#         cycles_csv_path=cycles_csv_path,
#         left_onsets=left_onsets,
#         right_onsets=right_onsets,
#         dance_mode_time_segments=dance_mode_time_segments,
#         figsize=(10, 3),
#         dpi=200,
#         use_window=True
#     )
    
#     # Store the data
#     if piece_type not in piece_dance_phases_kde:
#         piece_dance_phases_kde[piece_type] = []
#     piece_dance_phases_kde[piece_type].append(dance_phases_kde)
    
#     # Close the individual piece figure
#     plt.close(fig)


# save_path = os.path.join(base_output_dir, f"piece_dance_phases_kde_{dance_mode}.pkl")
# with open(save_path, 'wb') as f:
#     pickle.dump(piece_dance_phases_kde, f)
# print(f"\nSaved piece KDE data to: {save_path}")

In [11]:
m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]


save_path = os.path.join(base_output_dir, f"piece_dance_phases_kde_{dance_mode}.pkl")

with open(save_path, 'rb') as f:
    piece_dance_phases_kde = pickle.load(f)      # This was saved in the previous cell

# Create combined plots for each piece type
for piece_type in PIECE_TYPES:
    if piece_type in piece_dance_phases_kde:
        fig, ax = plot_combined_foot_stacked(
            piece_type=piece_type,
            dance_mode=dance_mode,
            dance_phases_kde_all=piece_dance_phases_kde[piece_type]
        )
        
        # Save the figure
        save_dir = os.path.join(base_output_dir, "dance_kde_by_piece", dance_mode)
        os.makedirs(save_dir, exist_ok=True)
        
        save_path = os.path.join(save_dir, f"{piece_type}_{dance_mode}_combined.png")
        plt.savefig(save_path, bbox_inches='tight')
        plt.close()

In [ ]:
m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]

load_drum_path = os.path.join(base_output_dir, f"piece_drum_phases_kde_{dance_mode}.pkl")
load_dance_path = os.path.join(base_output_dir, f"piece_dance_phases_kde_{dance_mode}.pkl")


with open(load_drum_path, 'rb') as f:        
    piece_drum_phases_kde = pickle.load(f) 
    

with open(load_dance_path, 'rb') as f:
    piece_dance_phases_kde = pickle.load(f)
    

 

In [ ]:
def plot_combined_drum_dance(piece_type, 
                             dance_mode, 
                             drum_phases_kde_all, 
                             dance_phases_kde_all, 
                             figsize=(10, 6), 
                             dpi=200,
                             ):
    """Create a single figure with two subplots: drum and dance"""
    
    # Create figure with two subplots
    fig, (ax2, ax1) = plt.subplots(2, 1, figsize=figsize, dpi=dpi)
    fig.tight_layout(pad=3.0)
    
    # Plot drum data (top subplot)
    onset_types = ['Dun', 'J1', 'J2']
    colors = ['#1f77b4', '#2ca02c', '#d62728']  # Blue, Green, Red
    
    # vertical stacking ranges for each type
    vertical_ranges = {
        'Dun': (1, 6),    # y from 1–6
        'J1':  (8, 13),   # y from 8–13
        'J2':  (15, 20),  # y from 15–20
    }

    combined_h = None
    combined_xx = None

    # Combine data for each onset type
    for onset_type, color in zip(onset_types, colors):
        # Combine phases and y_scaled from all pieces
        all_phases = []
        all_y_scaled = []
        segment_kde_h = None
        segment_kde_xx = None

        # Loop through all pieces' data
        for piece_data in drum_phases_kde_all:
            if onset_type in piece_data:
                # Combine phases and y_scaled
                all_phases.extend(piece_data[onset_type]["phases"])
                all_y_scaled.extend(piece_data[onset_type]["y_scaled"])
                
                # Accumulate KDE
                if segment_kde_h is None:
                    segment_kde_h = piece_data[onset_type]["kde_h"].copy()
                    segment_kde_xx = piece_data[onset_type]["kde_xx"].copy()
                else:
                    segment_kde_h += piece_data[onset_type]["kde_h"]

        if not all_phases:  # Skip if no data found
            continue

        # Convert lists to numpy arrays
        phases = np.array(all_phases)
        y_scaled = np.array(all_y_scaled)
            
        # Plot scatter with single color for each instrument
        ax1.scatter(phases * 400,
                   y_scaled,
                   s=5, alpha=0.6,
                   color=color,
                   label=onset_type)

        # accumulate KDE
        if combined_h is None:
            combined_h = segment_kde_h.copy()
            combined_xx = segment_kde_xx.copy()
        else:
            combined_h += segment_kde_h

    # Draw combined KDE at bottom (-5 to 0)
    if combined_h is not None:
        # normalize to 0–1
        combined_h = combined_h / np.max(combined_h)
        kde_scaled = -5 + (5 * combined_h)
        
        ax1.fill_between(combined_xx * 400,
                        -5, kde_scaled,
                        alpha=0.3, color='purple',
                        label='Combined KDE')

    # Add subdivision lines for drum
    for subdiv in range(1, 13):  # 12 subdivisions
        color = get_subdiv_color(subdiv)
        x_pos = ((subdiv-1) * 400) / 12
        
        if subdiv in [1, 4, 7, 10]:
            ax1.vlines(x_pos, -5.5, 20.5, color=color, linestyle='-', linewidth=1.5, alpha=0.7)
        else:
            ax1.vlines(x_pos, -5.5, 20.5, color=color, linestyle='--', linewidth=1, alpha=0.3)

    # Styling for drum plot
    xtick = [0, 100, 200, 300, 400]
    xtick_labels = [1, 2, 3, 4, 5]
    
    ax1.set_xlim(0, 400)
    ax1.set_xticks(xtick)
    ax1.set_xticklabels(xtick_labels)
    ax1.set_xlabel('Beat span')
    
    ax1.set_ylim(-5.5, 20.5)
    ax1.set_yticks([3, 10, 17])
    ax1.set_yticklabels(['Dun', 'J1', 'J2'])
    
    ax1.set_ylabel('Instrument')
    ax1.grid(True, alpha=0.3)
    ax1.set_xlim(-33, 400)

    # Title for drum plot
    ax1.set_title(f'{piece_type} | {dance_mode} | Drum Onsets', pad=10)
    ax1.legend(loc='upper left', framealpha=0.4, fontsize=6)

    # Plot dance data (bottom subplot)
    vertical_ranges = {
        'left': (1, 6),
        'right': (8, 13),
    }

    combined_phases = []
    
    # Combine data for each foot
    for foot_type, color in [('left', '#1f77b4'), ('right', '#d62728')]:
        # Combine phases and y_scaled from all pieces
        all_phases = []
        all_y_scaled = []
        segment_kde_h = None
        segment_kde_xx = None

        # Loop through all pieces' data
        for piece_data in dance_phases_kde_all:
            if foot_type in piece_data["phases"]:
                # Combine phases and y_scaled
                all_phases.extend(piece_data["phases"][foot_type])
                all_y_scaled.extend(piece_data["y_scaled"][foot_type])
                combined_phases.extend(piece_data["phases"][foot_type])

        if not all_phases:  # Skip if no data found
            continue

        # Convert lists to numpy arrays
        phases = np.array(all_phases)
        y_scaled = np.array(all_y_scaled)
            
        # Plot scatter with single color for each foot
        ax2.scatter(phases * 400,
                   y_scaled,
                   s=5, alpha=0.6,
                   color=color,
                   label=f'{foot_type.capitalize()} Foot')

    # Combined KDE at bottom using kde_estimate
    if len(combined_phases) > 0:
        kde_xx, kde_h = kde_estimate(np.array(combined_phases), SIG=0.01)
        
        # Only plot the region that maps to the x-axis
        mask = (kde_xx * 400 >= -33) & (kde_xx * 400 <= 400)
        kde_xx_plot = kde_xx[mask]
        kde_h_plot = kde_h[mask]
        
        if np.max(kde_h_plot) > 0:
            kde_scaled = -5 + (5 * kde_h_plot / np.max(kde_h_plot))
            ax2.fill_between(kde_xx_plot * 400, -5, kde_scaled, alpha=0.3, color='purple', label='Combined KDE')

    # Add subdivision lines for dance
    for subdiv in range(1, 13):
        color = get_subdiv_color(subdiv)
        x_pos = ((subdiv-1) * 400) / 12
        
        if subdiv in [1, 4, 7, 10]:
            ax2.vlines(x_pos, -5.5, 13.5, color=color, linestyle='-', linewidth=1.5, alpha=0.7)
        else:
            ax2.vlines(x_pos, -5.5, 13.5, color=color, linestyle='--', linewidth=1, alpha=0.3)

    # Styling for dance plot
    ax2.set_xticks(xtick)
    ax2.set_xticklabels(xtick_labels)
    ax2.set_xlim(-33, 400)
    ax2.set_xlabel('Beat span')
    
    ax2.set_ylim(-5.5, 13.5)
    ax2.set_yticks([3, 10])
    ax2.set_yticklabels(['LF', 'RF'])
    ax2.set_ylabel('Foot')
    ax2.grid(True, alpha=0.3)

    # Title for dance plot
    ax2.set_title(f'{piece_type} | {dance_mode} | Dance Onsets', pad=10)
    ax2.legend(loc='upper left', framealpha=0.4, fontsize=6)

    # Save the figure
    save_dir = os.path.join(base_output_dir, "drum_dance_kde", dance_mode)
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{piece_type}_{dance_mode}_combined.png")
    plt.savefig(save_path, bbox_inches='tight')
    plt.close()

In [6]:
# Main execution code
m_idx = 0
mode = ["group", "individual", "audience"]
dance_mode = mode[m_idx]

PIECE_TYPES = ["Suku", "Maraka", "Manjanin", "Wasulunka"]


# Load the saved data
load_drum_path = os.path.join(base_output_dir, f"piece_drum_phases_kde_{dance_mode}.pkl")
load_dance_path = os.path.join(base_output_dir, f"piece_dance_phases_kde_{dance_mode}.pkl")


with open(load_drum_path, 'rb') as f:        
    piece_drum_phases_kde = pickle.load(f) 

with open(load_dance_path, 'rb') as f:
    piece_dance_phases_kde = pickle.load(f)

# Create combined plots for each piece type
for piece_type in PIECE_TYPES:
    if piece_type in piece_drum_phases_kde and piece_type in piece_dance_phases_kde:
        plot_combined_drum_dance(
            piece_type=piece_type,
            dance_mode=dance_mode,
            drum_phases_kde_all=piece_drum_phases_kde[piece_type],
            dance_phases_kde_all=piece_dance_phases_kde[piece_type]
        )